In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [ ]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn import preprocessing

from gensim.models import Word2Vec

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, MaxPool1D, Dropout, Dense, GlobalMaxPooling1D, Embedding, Activation
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import files
upload=files.upload()

Saving Toxic.xlsx to Toxic.xlsx


In [ ]:
toxic=pd.read_excel('Toxic.xlsx')
toxic

,Text,Label
0,Does N.e.bodyelse Hear her Crazy ass Screamin ...,0
1,There are so many things that are incorrect wi...,0
2,3:26 hahah my boyfriend showed this song to me...,1
3,dick beyonce fuck y a ass hole you are truely ...,1
4,DongHaeTaemin and Kai ;A; luhansehun and bacon...,0
...,...,...
5020,It should be further noted that my penis looks...,1
5021,:Actually it was for changing the message I le...,1
5022,""" \n\n What the fuck """"urdu"""" pathan is doing ...",1
5023,:And this and this and this is blatant unsourc...,1


In [ ]:
toxic['Label'] = toxic['Label'].astype(int)
toxic['Text'] = toxic['Text'].astype(str)

In [ ]:
#case Folding
import re
def casefolding(title):
  title = str(title.lower())
  title = title.strip(" ")
  title = re.sub(r'[?|$|.|!_:"@&*^#)(-+,;}{%<>]','',title)
  return title
toxic['Text'] = toxic['Text'].apply(casefolding)
toxic

,Text,Label
0,does nebodyelse hear her crazy ass screamin ho...,0
1,there are so many things that are incorrect wi...,0
2,326 hahah my boyfriend showed this song to me ...,1
3,dick beyonce fuck y a ass hole you are truely ...,1
4,donghaetaemin and kai a luhansehun and bacon x...,0
...,...,...
5020,it should be further noted that my penis looks...,1
5021,actually it was for changing the message i lef...,1
5022,\n\n what the fuck urdu pathan is doing in th...,1
5023,and this and this and this is blatant unsource...,1


In [ ]:
# tokenize the data

token = Tokenizer(35789)
token.fit_on_texts(toxic['Text'])
text = token.texts_to_sequences(toxic['Text'])
text = pad_sequences(text)

In [ ]:
h_d = []
for i in toxic['Text']:
    h_d.append(i.split())
print(h_d[:2])

[['does', 'nebodyelse', 'hear', 'her', 'crazy', 'ass', 'screamin', 'hoe', 'everytime', 'she', "say's", 'stupid', 'hoe', 'lol', 'did', 'nebody', 'see', 'chris', 'browns', 'face', 'at', '204', 'wen', 'nicki', 'was', 'touchin', 'him', 'if', 'you', 'worked', 'out', 'we', "wouldn't", 'be', 'lookin', 'at', 'that', 'body', 'd', 'she', 'looks', 'so', 'tired', '/', 'she', 'has', 'ah', 'tatto', 'on', 'her', 'ass', 'wtf', 'is', 'this', 'oo', 'instagram', 'deze', "nutt's", 'in', 'yo', 'mouth', 'lmao', 'this', 'video/song', 'made', 'me', 'ah', 'lil', 'umm', 'in', 'the', 'mood', "we're", 'all', 'in', 'this', 'together', 'motherfucka', 'wow', 'you', 'have', 'an', 'amazing', 'voice', 'and', "i've", 'never', 'said', 'that', 'about', 'anyone'], ['there', 'are', 'so', 'many', 'things', 'that', 'are', 'incorrect', 'with', 'your', 'comment', "it's", 'unbelievable', 'guns', "don't", 'kill', 'people', 'a', 'gun', "doesn't", 'get', 'up', 'off', 'a', 'table', 'and', 'then', 'shoot', 'someone', "he's", 'not', '

In [ ]:
w2vc_model = Word2Vec(h_d, size=50, workers=32, min_count=1, window=3)
print(w2vc_model)

Word2Vec(vocab=55451, size=50, alpha=0.025)


In [ ]:
y = toxic['Label'].values

In [ ]:
# split the data into train test split

X_train, X_test, y_train, y_test = train_test_split(np.array(text), y, test_size=0.2, stratify = y)

In [ ]:
# build the model

model = Sequential()
model.add(w2vc_model.wv.get_keras_embedding(True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          2772550   
                                                                 
 dropout (Dropout)           (None, None, 50)          0         
                                                                 
 lstm (LSTM)                 (None, None, 50)          20200     
                                                                 
 global_max_pooling1d (Globa  (None, 50)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                        

In [ ]:
# compile and train model

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
126/126 [==============================] - 150s 1s/step - loss: 0.5935 - accuracy: 0.7102
Epoch 2/10
126/126 [==============================] - 153s 1s/step - loss: 0.5333 - accuracy: 0.7652
Epoch 3/10
126/126 [==============================] - 146s 1s/step - loss: 0.4922 - accuracy: 0.7891
Epoch 4/10
126/126 [==============================] - 146s 1s/step - loss: 0.4294 - accuracy: 0.8264
Epoch 5/10
126/126 [==============================] - 146s 1s/step - loss: 0.3603 - accuracy: 0.8567
Epoch 6/10
126/126 [==============================] - 145s 1s/step - loss: 0.2392 - accuracy: 0.9085
Epoch 7/10
126/126 [==============================] - 146s 1s/step - loss: 0.1563 - accuracy: 0.9423
Epoch 8/10
126/126 [==============================] - 146s 1s/step - loss: 0.0951 - accuracy: 0.9639
Epoch 9/10
126/126 [==============================] - 146s 1s/step - loss: 0.0662 - accuracy: 0.9754
Epoch 10/10
126/126 [==============================] - 147s 1s/step - loss: 0.0505 - accura

In [ ]:
#untuk LSTM (10 epoch)
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
y_pred = [round(i[0]) for i in model.predict(X_test)]
print("======= LSTM (10 Epoch)=======")
print('Precision: %.3f' % precision_score(y_test, y_pred))
print('Recall: %.3f' % recall_score(y_test, y_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('F1 Score: %.3f' % f1_score(y_test, y_pred))

======= LSTM (10 Epoch)=======
Precision: 0.728
Recall: 0.827
Accuracy: 0.778
F1 Score: 0.775


In [ ]:
# build the model

model = Sequential()
model.add(w2vc_model.wv.get_keras_embedding(True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 50)          2772550   
                                                                 
 dropout_2 (Dropout)         (None, None, 50)          0         
                                                                 
 lstm_1 (LSTM)               (None, None, 50)          20200     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 50)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                      

In [ ]:
# compile and train model

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=32, epochs=30)

Epoch 1/30
126/126 [==============================] - 150s 1s/step - loss: 0.5937 - accuracy: 0.7037
Epoch 2/30
126/126 [==============================] - 147s 1s/step - loss: 0.5462 - accuracy: 0.7520
Epoch 3/30
126/126 [==============================] - 148s 1s/step - loss: 0.5040 - accuracy: 0.7806
Epoch 4/30
126/126 [==============================] - 147s 1s/step - loss: 0.4371 - accuracy: 0.8184
Epoch 5/30
126/126 [==============================] - 148s 1s/step - loss: 0.3728 - accuracy: 0.8547
Epoch 6/30
126/126 [==============================] - 147s 1s/step - loss: 0.2442 - accuracy: 0.9075
Epoch 7/30
126/126 [==============================] - 148s 1s/step - loss: 0.1501 - accuracy: 0.9455
Epoch 8/30
126/126 [==============================] - 149s 1s/step - loss: 0.0912 - accuracy: 0.9672
Epoch 9/30
126/126 [==============================] - 147s 1s/step - loss: 0.0680 - accuracy: 0.9786
Epoch 10/30
126/126 [==============================] - 148s 1s/step - loss: 0.0425 - accura

In [ ]:
#untuk LSTM (30 epoch)
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
y_pred = [round(i[0]) for i in model.predict(X_test)]
print("======= LSTM (10 Epoch)=======")
print('Precision: %.3f' % precision_score(y_test, y_pred))
print('Recall: %.3f' % recall_score(y_test, y_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('F1 Score: %.3f' % f1_score(y_test, y_pred))

======= LSTM (10 Epoch)=======
Precision: 0.858
Recall: 0.680
Accuracy: 0.801
F1 Score: 0.759


In [ ]:
# build the model

model = Sequential()
model.add(w2vc_model.wv.get_keras_embedding(True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 50)          2772550   
                                                                 
 dropout_4 (Dropout)         (None, None, 50)          0         
                                                                 
 lstm_2 (LSTM)               (None, None, 50)          20200     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 50)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_5 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                      

In [ ]:
# compile and train model

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=32, epochs=50)

Epoch 1/50
126/126 [==============================] - 152s 1s/step - loss: 0.5912 - accuracy: 0.7114
Epoch 2/50
126/126 [==============================] - 151s 1s/step - loss: 0.5388 - accuracy: 0.7600
Epoch 3/50
126/126 [==============================] - 151s 1s/step - loss: 0.4967 - accuracy: 0.7826
Epoch 4/50
126/126 [==============================] - 151s 1s/step - loss: 0.4265 - accuracy: 0.8291
Epoch 5/50
126/126 [==============================] - 151s 1s/step - loss: 0.3699 - accuracy: 0.8540
Epoch 6/50
126/126 [==============================] - 153s 1s/step - loss: 0.2321 - accuracy: 0.9142
Epoch 7/50
126/126 [==============================] - 151s 1s/step - loss: 0.1315 - accuracy: 0.9552
Epoch 8/50
126/126 [==============================] - 150s 1s/step - loss: 0.1087 - accuracy: 0.9632
Epoch 9/50
126/126 [==============================] - 149s 1s/step - loss: 0.0747 - accuracy: 0.9764
Epoch 10/50
126/126 [==============================] - 150s 1s/step - loss: 0.0476 - accura

In [ ]:
#untuk LSTM (50 epoch)
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
y_pred = [round(i[0]) for i in model.predict(X_test)]
print("======= LSTM (10 Epoch)=======")
print('Precision: %.3f' % precision_score(y_test, y_pred))
print('Recall: %.3f' % recall_score(y_test, y_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('F1 Score: %.3f' % f1_score(y_test, y_pred))

======= LSTM (10 Epoch)=======
Precision: 0.816
Recall: 0.745
Accuracy: 0.805
F1 Score: 0.779


In [ ]:
test = ['I love you','Hate you']
token_text = pad_sequences(token.texts_to_sequences(test))
preds = [round(i[0]) for i in model.predict(token_text)]
for (text, pred) in zip(test, preds):
    label = 'toxic' if pred == 1.0 else 'Not toxic'
    print("{} - {}".format(text, label))

I love you - Not toxic
Hate you - toxic


In [ ]:
# naives bayes
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred  =  classifier.predict(X_test)


In [ ]:
print("======= Naives Bayes=======")
print('Precision: %.3f' % precision_score(y_test, y_pred))
print('Recall: %.3f' % recall_score(y_test, y_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('F1 Score: %.3f' % f1_score(y_test, y_pred))


======= Naives Bayes=======
Precision: 0.496
Recall: 0.942
Accuracy: 0.532
F1 Score: 0.650


In [ ]:
#SVM
from sklearn import svm
svm = svm.SVC(kernel='linear')
svm.fit(X_train, y_train) 
y_pred = svm.predict(X_test)

In [ ]:
print("======= SVM=======")
print('Precision: %.3f' % precision_score(y_test, y_pred))
print('Recall: %.3f' % recall_score(y_test, y_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('F1 Score: %.3f' % f1_score(y_test, y_pred))

In [ ]:
# decision tree

from sklearn.tree import DecisionTreeClassifier
dsc = DecisionTreeClassifier()
dsc = dsc.fit(X_train,y_train)
y_pred = dsc.predict(X_test)

In [ ]:
print("======= Decision Tree=======")
print('Precision: %.3f' % precision_score(y_test, y_pred))
print('Recall: %.3f' % recall_score(y_test, y_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('F1 Score: %.3f' % f1_score(y_test, y_pred))

======= Decision Tree=======
Precision: 0.634
Recall: 0.648
Accuracy: 0.666
F1 Score: 0.641


In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
parameters = {'n_neighbors':[3, 5, 7,9]}
knn = KNeighborsClassifier()
classifier = GridSearchCV(knn, parameters)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)


In [ ]:
print('The parameters combination that would give best accuracy is : ')
print(classifier.best_params_)

The parameters combination that would give best accuracy is : 
{'n_neighbors': 7}


In [ ]:
print("======= KNN=======")
print('Precision: %.3f' % precision_score(y_test, y_pred))
print('Recall: %.3f' % recall_score(y_test, y_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('F1 Score: %.3f' % f1_score(y_test, y_pred))

======= KNN=======
Precision: 0.561
Recall: 0.870
Accuracy: 0.626
F1 Score: 0.682


In [ ]:
from sklearn.neural_network import MLPClassifier
parameters = {'solver': ['lbfgs', 'adam', 'sgd'], 'max_iter': [10,20,30 ]}
mlp = MLPClassifier()
classifier = GridSearchCV(mlp, parameters)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REAC

In [ ]:
print('The parameters combination that would give best accuracy is : ')
print(classifier.best_params_)

The parameters combination that would give best accuracy is : 
{'max_iter': 20, 'solver': 'adam'}


In [ ]:
print("======= MLP=======")
print('Precision: %.3f' % precision_score(y_test, y_pred))
print('Recall: %.3f' % recall_score(y_test, y_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('F1 Score: %.3f' % f1_score(y_test, y_pred))

======= MLP=======
Precision: 0.587
Recall: 0.577
Accuracy: 0.618
F1 Score: 0.582
